In [202]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import time
import math
import warnings
warnings.filterwarnings("ignore")
import glob

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (25,10)
mpl.rcParams['axes.grid'] = False

In [203]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization, UtilityFunction
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [204]:
df = pd.read_csv('https://raw.githubusercontent.com/Mohammad-Kaosain-Akbar/NILM-two-years-dataset/main/Data_Greener_all.csv')
df['Time']= pd.to_datetime(df['Time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Time               43824 non-null  datetime64[ns]
 1   Ventilation        43824 non-null  float64       
 2   Sokets plug        43824 non-null  float64       
 3   Lighting           43824 non-null  float64       
 4   Other electricity  43824 non-null  float64       
 5   Cooling            43824 non-null  float64       
 6   Heating            43824 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 2.3 MB


In [205]:
df

,Time,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating
0,2017-01-03 13:00:00,27.4,41.8,53.7,89.6,18.60,498.0
1,2017-01-03 14:00:00,21.6,37.5,50.9,64.6,61.90,500.0
2,2017-01-03 15:00:00,18.5,37.4,60.1,64.2,16.50,480.0
3,2017-01-03 16:00:00,29.7,38.0,52.6,43.2,19.80,390.0
4,2017-01-03 17:00:00,16.7,38.1,56.7,42.4,15.20,373.0
...,...,...,...,...,...,...,...
43819,2022-01-03 08:00:00,59.3,17.9,16.6,115.0,12.70,344.0
43820,2022-01-03 09:00:00,71.9,20.6,33.1,149.0,11.90,399.0
43821,2022-01-03 10:00:00,90.5,30.9,47.8,157.0,12.90,397.0
43822,2022-01-03 11:00:00,94.0,33.9,52.2,155.0,15.40,345.0


In [206]:
Total = df["Ventilation"] + df["Sokets plug"] + df["Lighting"] + df["Other electricity"] + df["Cooling"] + df["Heating"]
# https://www.kite.com/python/answers/how-to-sum-two-columns-in-a-pandas-dataframe-in-python

df["Total"] = Total
df

,Time,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total
0,2017-01-03 13:00:00,27.4,41.8,53.7,89.6,18.60,498.0,729.10
1,2017-01-03 14:00:00,21.6,37.5,50.9,64.6,61.90,500.0,736.50
2,2017-01-03 15:00:00,18.5,37.4,60.1,64.2,16.50,480.0,676.70
3,2017-01-03 16:00:00,29.7,38.0,52.6,43.2,19.80,390.0,573.30
4,2017-01-03 17:00:00,16.7,38.1,56.7,42.4,15.20,373.0,542.10
...,...,...,...,...,...,...,...,...
43819,2022-01-03 08:00:00,59.3,17.9,16.6,115.0,12.70,344.0,565.50
43820,2022-01-03 09:00:00,71.9,20.6,33.1,149.0,11.90,399.0,685.50
43821,2022-01-03 10:00:00,90.5,30.9,47.8,157.0,12.90,397.0,736.10
43822,2022-01-03 11:00:00,94.0,33.9,52.2,155.0,15.40,345.0,695.50


In [207]:
df['Year'] = df['Time'].dt.year
df['Month']=df['Time'].dt.month
df['Quarter_of_year']=df['Time'].dt.quarter
df['Day_of_week']=df['Time'].dt.day_of_week
df['Week_of_year']=df['Time'].dt.week
df['Day']=df['Time'].dt.day
df['Hour']=df['Time'].dt.hour

df.drop('Time', inplace=True, axis=1)
df

,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total,Year,Month,Quarter_of_year,Day_of_week,Week_of_year,Day,Hour
0,27.4,41.8,53.7,89.6,18.60,498.0,729.10,2017,1,1,1,1,3,13
1,21.6,37.5,50.9,64.6,61.90,500.0,736.50,2017,1,1,1,1,3,14
2,18.5,37.4,60.1,64.2,16.50,480.0,676.70,2017,1,1,1,1,3,15
3,29.7,38.0,52.6,43.2,19.80,390.0,573.30,2017,1,1,1,1,3,16
4,16.7,38.1,56.7,42.4,15.20,373.0,542.10,2017,1,1,1,1,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,59.3,17.9,16.6,115.0,12.70,344.0,565.50,2022,1,1,0,1,3,8
43820,71.9,20.6,33.1,149.0,11.90,399.0,685.50,2022,1,1,0,1,3,9
43821,90.5,30.9,47.8,157.0,12.90,397.0,736.10,2022,1,1,0,1,3,10
43822,94.0,33.9,52.2,155.0,15.40,345.0,695.50,2022,1,1,0,1,3,11


In [208]:
# df.iloc[df['Ventilation'].idxmax()]

In [209]:
df_train = df.loc[(df['Year'].isin([2017,2018]))]
df_test = df.loc[(df['Year'].isin([2019]))]

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_train

,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total,Year,Month,Quarter_of_year,Day_of_week,Week_of_year,Day,Hour
0,27.4,41.8,53.70,89.60,18.60,498.0,729.10,2017,1,1,1,1,3,13
1,21.6,37.5,50.90,64.60,61.90,500.0,736.50,2017,1,1,1,1,3,14
2,18.5,37.4,60.10,64.20,16.50,480.0,676.70,2017,1,1,1,1,3,15
3,29.7,38.0,52.60,43.20,19.80,390.0,573.30,2017,1,1,1,1,3,16
4,16.7,38.1,56.70,42.40,15.20,373.0,542.10,2017,1,1,1,1,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17454,0.0,11.7,7.50,11.60,8.69,250.0,289.49,2018,12,4,0,1,31,19
17455,0.0,10.9,9.38,10.50,8.75,234.0,273.53,2018,12,4,0,1,31,20
17456,0.0,12.0,8.06,7.50,10.40,233.0,270.96,2018,12,4,0,1,31,21
17457,0.0,10.3,6.75,9.63,10.90,234.0,271.58,2018,12,4,0,1,31,22


In [210]:
df_test

,Ventilation,Sokets plug,Lighting,Other electricity,Cooling,Heating,Total,Year,Month,Quarter_of_year,Day_of_week,Week_of_year,Day,Hour
0,0.00,9.44,1.000,13.00,8.84,250.0,282.280,2019,1,1,1,1,1,0
1,0.00,11.30,2.310,8.34,9.78,217.0,248.730,2019,1,1,1,1,1,1
2,0.00,10.30,1.690,11.60,8.75,234.0,266.340,2019,1,1,1,1,1,2
3,4.50,10.50,0.375,7.56,9.06,233.0,264.995,2019,1,1,1,1,1,3
4,4.44,9.31,0.625,8.88,8.75,267.0,299.005,2019,1,1,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,10.30,11.30,13.600,458.00,0.00,222.0,715.200,2019,12,4,1,1,31,19
8756,9.50,11.90,13.400,461.00,0.00,206.0,701.800,2019,12,4,1,1,31,20
8757,8.25,10.10,10.100,458.00,0.00,205.0,691.450,2019,12,4,1,1,31,21
8758,10.50,10.40,10.600,457.00,0.00,222.0,710.500,2019,12,4,1,1,31,22


In [211]:
# X_train1 = df_train[['Total','Year', 'Month', 'Day', 'Hour']].values
# y_train1 = df_train[['Heating']].values
# X_test1 = df_test[['Total','Year', 'Month', 'Day', 'Hour']].values
# y_test1 = df_test[['Heating']].values
# print(X_train1.shape, y_train1.shape, X_test1.shape, y_test1.shape)

In [212]:
X_train1 = df_train[['Total','Year', 'Month', 'Day', 'Hour']].values
y_train1 = df_train[['Ventilation','Sokets plug', 'Lighting', 'Other electricity', 'Cooling', 'Heating']].values
X_test1 = df_test[['Total','Year', 'Month', 'Day', 'Hour']].values
y_test1 = df_test[['Ventilation','Sokets plug', 'Lighting', 'Other electricity', 'Cooling', 'Heating']].values
print(X_train1.shape, y_train1.shape, X_test1.shape, y_test1.shape)

(17459, 5) (17459, 6) (8760, 5) (8760, 6)


In [213]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train1 = sc.fit_transform(X_train1)
X_test1 = sc.fit_transform(X_test1)

y_train1 = sc.fit_transform(y_train1)
y_test1 = sc.fit_transform(y_test1)

In [214]:
from sklearn.multioutput import MultiOutputRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import RANSACRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [215]:
# bayes_opt requires this to be a dictionary.
# gp_params = {"alpha": 1e-10}

pbounds = {"learning_rate": [.1, 10],    # Gradient Boosting Regressor
           "min_samples_leaf1": [1, 20],
           "n_estimators1": [1, 20],
           "subsample": [0.1,1],
           "max_depth1": [1,50],
           "max_leaf_nodes1": [2,50],
           
           "n_estimators2": [50,200],   # Random Forest Regressor
           "max_depth2": [1,100],
           "min_samples_leaf2": [1,20],
           "max_leaf_nodes2" : [2,10],

           "n_neighbors" : [5,100],     
           "leaf_size": [30, 60],
           "p": [1,2],

           "min_samples": [2,100],
           "max_trials": [50,200],
           "stop_probability": [0,1],

          #  "boosting_type": ['gbdt', 'dart', 'gross', 'rf'],
           "num_leaves" :[31, 150],
           "max_depth3": [-1, 150],
           "n_estimators3": [50, 150],
           "min_split_gain": [20, 150],
           "min_child_samples": [20, 150],
           
          #  "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
          #  "splitter": ['best', 'random'],
           "max_depth4": [1,150],
           "min_samples_split": [2, 200],
           "min_weight_fraction_leaf": [0.0, 0.5],
           "min_impurity_decrease": [0,100],
           "max_leaf_nodes": [1,100]
          #  "max_features": ['auto', 'sqrt', 'log2']
      
           }

# **Model For the Whole dataset**

In [216]:

def black_box_function0(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

    l=[]
# GRADIENT BOOSTING REGRESSOR

    model1 = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           ))
    model1.fit(X_train1, y_train1)
    pred1 = model1.predict(X_test1)



# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train1)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train1)
    pred3 = model3.predict(X_test1)


# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train1)
    pred4 = model4.predict(X_test1)


# LGBM REGRESSOR

    model5 = MultiOutputRegressor(LGBMRegressor(
                                                num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                ))
    


    model5.fit(X_train1, y_train1)
    pred5 = model5.predict(X_test1)


# DECISION TREE REGRESSOR

    model6 = MultiOutputRegressor(DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                ))
    


    model6.fit(X_train1, y_train1)
    pred6 = model6.predict(X_test1)


    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6
    
    r2 = r2_score(y_test1,pred1)
    l.append(r2)
    r2 = r2_score(y_test1,pred2)
    l.append(r2)    
    r2 = r2_score(y_test1,pred3)
    l.append(r2)
    r2 = r2_score(y_test1,pred4)
    l.append(r2)
    r2 = r2_score(y_test1,pred5)
    l.append(r2)
    r2 = r2_score(y_test1,pred6)
    l.append(r2)
    r2 = r2_score(y_test1,pred)
    l.append(r2)
    r2 = max(l)

    r2 = round(r2, 3)
    return r2


In [217]:
optimizer = BayesianOptimization(f = black_box_function0, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.405     | 59.01     | 5.518     | 48.66     | 71.77     | 104.4     | 33.2      | 97.65     | 2.299     | 4.024     | 115.2     | 121.3     | 19.77     | 86.57     | 19.68     | 4.113     | 120.3     | 21.17     | 0.1933    | 1.839     | 193.5     | 93.61     | 95.15     | 124.6     | 1.866     | 0.1732    

In [218]:
print(optimizer.max)

{'target': 0.405, 'params': {'leaf_size': 59.0108951704103, 'learning_rate': 5.5175992668396505, 'max_depth1': 48.66153363827933, 'max_depth2': 71.7667833737621, 'max_depth3': 104.3570525141879, 'max_depth4': 33.19733484147608, 'max_leaf_nodes': 97.65117102284793, 'max_leaf_nodes1': 2.2990522498203134, 'max_leaf_nodes2': 4.023858899067552, 'max_trials': 115.21872986066687, 'min_child_samples': 121.31977983318782, 'min_impurity_decrease': 19.768507460025308, 'min_samples': 86.57333708872378, 'min_samples_leaf1': 19.684612866330944, 'min_samples_leaf2': 4.113002586689276, 'min_samples_split': 120.27212089870612, 'min_split_gain': 21.168192696782146, 'min_weight_fraction_leaf': 0.1932856413218147, 'n_estimators1': 1.839041100698492, 'n_estimators2': 193.49794515713538, 'n_estimators3': 93.61466468797977, 'n_neighbors': 95.15284414424846, 'num_leaves': 124.57041232627225, 'p': 1.8662892985816986, 'stop_probability': 0.17316542149594816, 'subsample': 0.16745372831179633}}


In [219]:
parameters_whole = optimizer.max["params"]
parameters_whole

{'leaf_size': 59.0108951704103,
 'learning_rate': 5.5175992668396505,
 'max_depth1': 48.66153363827933,
 'max_depth2': 71.7667833737621,
 'max_depth3': 104.3570525141879,
 'max_depth4': 33.19733484147608,
 'max_leaf_nodes': 97.65117102284793,
 'max_leaf_nodes1': 2.2990522498203134,
 'max_leaf_nodes2': 4.023858899067552,
 'max_trials': 115.21872986066687,
 'min_child_samples': 121.31977983318782,
 'min_impurity_decrease': 19.768507460025308,
 'min_samples': 86.57333708872378,
 'min_samples_leaf1': 19.684612866330944,
 'min_samples_leaf2': 4.113002586689276,
 'min_samples_split': 120.27212089870612,
 'min_split_gain': 21.168192696782146,
 'min_weight_fraction_leaf': 0.1932856413218147,
 'n_estimators1': 1.839041100698492,
 'n_estimators2': 193.49794515713538,
 'n_estimators3': 93.61466468797977,
 'n_neighbors': 95.15284414424846,
 'num_leaves': 124.57041232627225,
 'p': 1.8662892985816986,
 'stop_probability': 0.17316542149594816,
 'subsample': 0.16745372831179633}

# **For Appliance 1 (Ventilation)**

In [220]:

def black_box_function1(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

    l = []
    y_train = y_train1[:,[0,]]
    y_train = np.asarray(y_train).reshape(-1)
# GRADIENT BOOSTING REGRESSOR

    model1 = GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           )
    
    model1.fit(X_train1, y_train)
    pred1 = model1.predict(X_test1)
    
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train)
    pred3 = model3.predict(X_test1)



# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train)
    pred4 = model4.predict(X_test1)



# LGBM REGRESSOR

    model5 = LGBMRegressor(num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                )
    


    model5.fit(X_train1, y_train)
    pred5 = model5.predict(X_test1)



# DECISION TREE REGRESSOR

    model6 = DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                )
    


    model6.fit(X_train1, y_train)
    pred6 = model6.predict(X_test1)



    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[0,]]
    test = np.asarray(test).reshape(-1)

    r2 = r2_score(test,pred1)
    l.append(r2)
    r2 = r2_score(test,pred2)
    l.append(r2)    
    r2 = r2_score(test,pred3)
    l.append(r2)
    r2 = r2_score(test,pred4)
    l.append(r2)
    r2 = r2_score(test,pred5)
    l.append(r2)
    r2 = r2_score(test,pred6)
    l.append(r2)
    r2 = r2_score(test,pred)
    l.append(r2)
    r2 = max(l)

    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [221]:
optimizer1 = BayesianOptimization(f = black_box_function1, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer1.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.093     | 59.01     | 5.518     | 48.66     | 71.77     | 104.4     | 33.2      | 97.65     | 2.299     | 4.024     | 115.2     | 121.3     | 19.77     | 86.57     | 19.68     | 4.113     | 120.3     | 21.17     | 0.1933    | 1.839     | 193.5     | 93.61     | 95.15     | 124.6     | 1.866     | 0.1732    

In [222]:
print(optimizer1.max)

{'target': 0.248, 'params': {'leaf_size': 33.39573998671889, 'learning_rate': 2.3841879334875045, 'max_depth1': 38.01694930883292, 'max_depth2': 11.570119804765044, 'max_depth3': 90.19373772565454, 'max_depth4': 142.01858349838741, 'max_leaf_nodes': 49.46969763289307, 'max_leaf_nodes1': 4.136393938084989, 'max_leaf_nodes2': 2.6631253117456346, 'max_trials': 185.50195874479385, 'min_child_samples': 134.1916425004709, 'min_impurity_decrease': 65.51939439569226, 'min_samples': 56.841948929049295, 'min_samples_leaf1': 14.89851105347434, 'min_samples_leaf2': 19.59048534615314, 'min_samples_split': 170.20687665162228, 'min_split_gain': 58.053434811506115, 'min_weight_fraction_leaf': 0.458477601156334, 'n_estimators1': 3.3680479746733027, 'n_estimators2': 74.32055679491002, 'n_estimators3': 87.3917861911129, 'n_neighbors': 44.02355818971096, 'num_leaves': 32.29685150575669, 'p': 1.5561605760810528, 'stop_probability': 0.9295883845940125, 'subsample': 0.6533904038111469}}


In [223]:
parameters_vent = optimizer1.max["params"]
parameters_vent

{'leaf_size': 33.39573998671889,
 'learning_rate': 2.3841879334875045,
 'max_depth1': 38.01694930883292,
 'max_depth2': 11.570119804765044,
 'max_depth3': 90.19373772565454,
 'max_depth4': 142.01858349838741,
 'max_leaf_nodes': 49.46969763289307,
 'max_leaf_nodes1': 4.136393938084989,
 'max_leaf_nodes2': 2.6631253117456346,
 'max_trials': 185.50195874479385,
 'min_child_samples': 134.1916425004709,
 'min_impurity_decrease': 65.51939439569226,
 'min_samples': 56.841948929049295,
 'min_samples_leaf1': 14.89851105347434,
 'min_samples_leaf2': 19.59048534615314,
 'min_samples_split': 170.20687665162228,
 'min_split_gain': 58.053434811506115,
 'min_weight_fraction_leaf': 0.458477601156334,
 'n_estimators1': 3.3680479746733027,
 'n_estimators2': 74.32055679491002,
 'n_estimators3': 87.3917861911129,
 'n_neighbors': 44.02355818971096,
 'num_leaves': 32.29685150575669,
 'p': 1.5561605760810528,
 'stop_probability': 0.9295883845940125,
 'subsample': 0.6533904038111469}

# **Appliance 2 (Socket Plugs)**

In [224]:

def black_box_function2(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

    l = []
    y_train = y_train1[:,[1,]]
    y_train = np.asarray(y_train).reshape(-1)
# GRADIENT BOOSTING REGRESSOR

    model1 = GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           )
    
    model1.fit(X_train1, y_train)
    pred1 = model1.predict(X_test1)
    
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train)
    pred3 = model3.predict(X_test1)



# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train)
    pred4 = model4.predict(X_test1)



# LGBM REGRESSOR

    model5 = LGBMRegressor(num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                )
    


    model5.fit(X_train1, y_train)
    pred5 = model5.predict(X_test1)



# DECISION TREE REGRESSOR

    model6 = DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                )
    


    model6.fit(X_train1, y_train)
    pred6 = model6.predict(X_test1)



    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[1,]]
    test = np.asarray(test).reshape(-1)

    r2 = r2_score(test,pred1)
    l.append(r2)
    r2 = r2_score(test,pred2)
    l.append(r2)    
    r2 = r2_score(test,pred3)
    l.append(r2)
    r2 = r2_score(test,pred4)
    l.append(r2)
    r2 = r2_score(test,pred5)
    l.append(r2)
    r2 = r2_score(test,pred6)
    l.append(r2)
    r2 = r2_score(test,pred)
    l.append(r2)
    r2 = max(l)

    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [225]:
optimizer2 = BayesianOptimization(f = black_box_function2, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer2.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | -0.075    | 59.01     | 5.518     | 48.66     | 71.77     | 104.4     | 33.2      | 97.65     | 2.299     | 4.024     | 115.2     | 121.3     | 19.77     | 86.57     | 19.68     | 4.113     | 120.3     | 21.17     | 0.1933    | 1.839     | 193.5     | 93.61     | 95.15     | 124.6     | 1.866     | 0.1732    

In [226]:
print(optimizer2.max)

{'target': -0.075, 'params': {'leaf_size': 59.0108951704103, 'learning_rate': 5.5175992668396505, 'max_depth1': 48.66153363827933, 'max_depth2': 71.7667833737621, 'max_depth3': 104.3570525141879, 'max_depth4': 33.19733484147608, 'max_leaf_nodes': 97.65117102284793, 'max_leaf_nodes1': 2.2990522498203134, 'max_leaf_nodes2': 4.023858899067552, 'max_trials': 115.21872986066687, 'min_child_samples': 121.31977983318782, 'min_impurity_decrease': 19.768507460025308, 'min_samples': 86.57333708872378, 'min_samples_leaf1': 19.684612866330944, 'min_samples_leaf2': 4.113002586689276, 'min_samples_split': 120.27212089870612, 'min_split_gain': 21.168192696782146, 'min_weight_fraction_leaf': 0.1932856413218147, 'n_estimators1': 1.839041100698492, 'n_estimators2': 193.49794515713538, 'n_estimators3': 93.61466468797977, 'n_neighbors': 95.15284414424846, 'num_leaves': 124.57041232627225, 'p': 1.8662892985816986, 'stop_probability': 0.17316542149594816, 'subsample': 0.16745372831179633}}


In [227]:
parameters_soc = optimizer2.max["params"]
parameters_soc

{'leaf_size': 59.0108951704103,
 'learning_rate': 5.5175992668396505,
 'max_depth1': 48.66153363827933,
 'max_depth2': 71.7667833737621,
 'max_depth3': 104.3570525141879,
 'max_depth4': 33.19733484147608,
 'max_leaf_nodes': 97.65117102284793,
 'max_leaf_nodes1': 2.2990522498203134,
 'max_leaf_nodes2': 4.023858899067552,
 'max_trials': 115.21872986066687,
 'min_child_samples': 121.31977983318782,
 'min_impurity_decrease': 19.768507460025308,
 'min_samples': 86.57333708872378,
 'min_samples_leaf1': 19.684612866330944,
 'min_samples_leaf2': 4.113002586689276,
 'min_samples_split': 120.27212089870612,
 'min_split_gain': 21.168192696782146,
 'min_weight_fraction_leaf': 0.1932856413218147,
 'n_estimators1': 1.839041100698492,
 'n_estimators2': 193.49794515713538,
 'n_estimators3': 93.61466468797977,
 'n_neighbors': 95.15284414424846,
 'num_leaves': 124.57041232627225,
 'p': 1.8662892985816986,
 'stop_probability': 0.17316542149594816,
 'subsample': 0.16745372831179633}

# **Appliance 3 (Lighting)**

In [228]:

def black_box_function3(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

    l = []
    y_train = y_train1[:,[2,]]
    y_train = np.asarray(y_train).reshape(-1)
# GRADIENT BOOSTING REGRESSOR

    model1 = GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           )
    
    model1.fit(X_train1, y_train)
    pred1 = model1.predict(X_test1)
    
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train)
    pred3 = model3.predict(X_test1)



# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train)
    pred4 = model4.predict(X_test1)



# LGBM REGRESSOR

    model5 = LGBMRegressor(num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                )
    


    model5.fit(X_train1, y_train)
    pred5 = model5.predict(X_test1)



# DECISION TREE REGRESSOR

    model6 = DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                )
    


    model6.fit(X_train1, y_train)
    pred6 = model6.predict(X_test1)



    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[2,]]
    test = np.asarray(test).reshape(-1)

    r2 = r2_score(test,pred1)
    l.append(r2)
    r2 = r2_score(test,pred2)
    l.append(r2)    
    r2 = r2_score(test,pred3)
    l.append(r2)
    r2 = r2_score(test,pred4)
    l.append(r2)
    r2 = r2_score(test,pred5)
    l.append(r2)
    r2 = r2_score(test,pred6)
    l.append(r2)
    r2 = r2_score(test,pred)
    l.append(r2)
    r2 = max(l)

    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [229]:
optimizer3 = BayesianOptimization(f = black_box_function3, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer3.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.292     | 59.01     | 5.518     | 48.66     | 71.77     | 104.4     | 33.2      | 97.65     | 2.299     | 4.024     | 115.2     | 121.3     | 19.77     | 86.57     | 19.68     | 4.113     | 120.3     | 21.17     | 0.1933    | 1.839     | 193.5     | 93.61     | 95.15     | 124.6     | 1.866     | 0.1732    

In [230]:
print(optimizer3.max)

{'target': 0.292, 'params': {'leaf_size': 59.0108951704103, 'learning_rate': 5.5175992668396505, 'max_depth1': 48.66153363827933, 'max_depth2': 71.7667833737621, 'max_depth3': 104.3570525141879, 'max_depth4': 33.19733484147608, 'max_leaf_nodes': 97.65117102284793, 'max_leaf_nodes1': 2.2990522498203134, 'max_leaf_nodes2': 4.023858899067552, 'max_trials': 115.21872986066687, 'min_child_samples': 121.31977983318782, 'min_impurity_decrease': 19.768507460025308, 'min_samples': 86.57333708872378, 'min_samples_leaf1': 19.684612866330944, 'min_samples_leaf2': 4.113002586689276, 'min_samples_split': 120.27212089870612, 'min_split_gain': 21.168192696782146, 'min_weight_fraction_leaf': 0.1932856413218147, 'n_estimators1': 1.839041100698492, 'n_estimators2': 193.49794515713538, 'n_estimators3': 93.61466468797977, 'n_neighbors': 95.15284414424846, 'num_leaves': 124.57041232627225, 'p': 1.8662892985816986, 'stop_probability': 0.17316542149594816, 'subsample': 0.16745372831179633}}


In [231]:
parameters_light = optimizer3.max["params"]
parameters_light

{'leaf_size': 59.0108951704103,
 'learning_rate': 5.5175992668396505,
 'max_depth1': 48.66153363827933,
 'max_depth2': 71.7667833737621,
 'max_depth3': 104.3570525141879,
 'max_depth4': 33.19733484147608,
 'max_leaf_nodes': 97.65117102284793,
 'max_leaf_nodes1': 2.2990522498203134,
 'max_leaf_nodes2': 4.023858899067552,
 'max_trials': 115.21872986066687,
 'min_child_samples': 121.31977983318782,
 'min_impurity_decrease': 19.768507460025308,
 'min_samples': 86.57333708872378,
 'min_samples_leaf1': 19.684612866330944,
 'min_samples_leaf2': 4.113002586689276,
 'min_samples_split': 120.27212089870612,
 'min_split_gain': 21.168192696782146,
 'min_weight_fraction_leaf': 0.1932856413218147,
 'n_estimators1': 1.839041100698492,
 'n_estimators2': 193.49794515713538,
 'n_estimators3': 93.61466468797977,
 'n_neighbors': 95.15284414424846,
 'num_leaves': 124.57041232627225,
 'p': 1.8662892985816986,
 'stop_probability': 0.17316542149594816,
 'subsample': 0.16745372831179633}

# **Appliance 4 (Other Electricity)**

In [232]:

def black_box_function4(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

    l = []
    y_train = y_train1[:,[3,]]
    y_train = np.asarray(y_train).reshape(-1)
# GRADIENT BOOSTING REGRESSOR

    model1 = GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           )
    
    model1.fit(X_train1, y_train)
    pred1 = model1.predict(X_test1)
    
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train)
    pred3 = model3.predict(X_test1)



# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train)
    pred4 = model4.predict(X_test1)



# LGBM REGRESSOR

    model5 = LGBMRegressor(num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                )
    


    model5.fit(X_train1, y_train)
    pred5 = model5.predict(X_test1)



# DECISION TREE REGRESSOR

    model6 = DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                )
    


    model6.fit(X_train1, y_train)
    pred6 = model6.predict(X_test1)



    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[3,]]
    test = np.asarray(test).reshape(-1)

    r2 = r2_score(test,pred1)
    l.append(r2)
    r2 = r2_score(test,pred2)
    l.append(r2)    
    r2 = r2_score(test,pred3)
    l.append(r2)
    r2 = r2_score(test,pred4)
    l.append(r2)
    r2 = r2_score(test,pred5)
    l.append(r2)
    r2 = r2_score(test,pred6)
    l.append(r2)
    r2 = r2_score(test,pred)
    l.append(r2)
    r2 = max(l)

    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [233]:
optimizer4 = BayesianOptimization(f = black_box_function4, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer4.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | -0.61     | 59.01     | 5.518     | 48.66     | 71.77     | 104.4     | 33.2      | 97.65     | 2.299     | 4.024     | 115.2     | 121.3     | 19.77     | 86.57     | 19.68     | 4.113     | 120.3     | 21.17     | 0.1933    | 1.839     | 193.5     | 93.61     | 95.15     | 124.6     | 1.866     | 0.1732    

In [234]:
print(optimizer4.max)

{'target': -0.61, 'params': {'leaf_size': 59.0108951704103, 'learning_rate': 5.5175992668396505, 'max_depth1': 48.66153363827933, 'max_depth2': 71.7667833737621, 'max_depth3': 104.3570525141879, 'max_depth4': 33.19733484147608, 'max_leaf_nodes': 97.65117102284793, 'max_leaf_nodes1': 2.2990522498203134, 'max_leaf_nodes2': 4.023858899067552, 'max_trials': 115.21872986066687, 'min_child_samples': 121.31977983318782, 'min_impurity_decrease': 19.768507460025308, 'min_samples': 86.57333708872378, 'min_samples_leaf1': 19.684612866330944, 'min_samples_leaf2': 4.113002586689276, 'min_samples_split': 120.27212089870612, 'min_split_gain': 21.168192696782146, 'min_weight_fraction_leaf': 0.1932856413218147, 'n_estimators1': 1.839041100698492, 'n_estimators2': 193.49794515713538, 'n_estimators3': 93.61466468797977, 'n_neighbors': 95.15284414424846, 'num_leaves': 124.57041232627225, 'p': 1.8662892985816986, 'stop_probability': 0.17316542149594816, 'subsample': 0.16745372831179633}}


In [235]:
parameters_other = optimizer4.max["params"]
parameters_other

{'leaf_size': 59.0108951704103,
 'learning_rate': 5.5175992668396505,
 'max_depth1': 48.66153363827933,
 'max_depth2': 71.7667833737621,
 'max_depth3': 104.3570525141879,
 'max_depth4': 33.19733484147608,
 'max_leaf_nodes': 97.65117102284793,
 'max_leaf_nodes1': 2.2990522498203134,
 'max_leaf_nodes2': 4.023858899067552,
 'max_trials': 115.21872986066687,
 'min_child_samples': 121.31977983318782,
 'min_impurity_decrease': 19.768507460025308,
 'min_samples': 86.57333708872378,
 'min_samples_leaf1': 19.684612866330944,
 'min_samples_leaf2': 4.113002586689276,
 'min_samples_split': 120.27212089870612,
 'min_split_gain': 21.168192696782146,
 'min_weight_fraction_leaf': 0.1932856413218147,
 'n_estimators1': 1.839041100698492,
 'n_estimators2': 193.49794515713538,
 'n_estimators3': 93.61466468797977,
 'n_neighbors': 95.15284414424846,
 'num_leaves': 124.57041232627225,
 'p': 1.8662892985816986,
 'stop_probability': 0.17316542149594816,
 'subsample': 0.16745372831179633}

# **Appliance 5 (Cooling)**

In [236]:

def black_box_function5(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

    l = []
    y_train = y_train1[:,[4,]]
    y_train = np.asarray(y_train).reshape(-1)
# GRADIENT BOOSTING REGRESSOR

    model1 = GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           )
    
    model1.fit(X_train1, y_train)
    pred1 = model1.predict(X_test1)
    
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train)
    pred3 = model3.predict(X_test1)



# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train)
    pred4 = model4.predict(X_test1)



# LGBM REGRESSOR

    model5 = LGBMRegressor(num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                )
    


    model5.fit(X_train1, y_train)
    pred5 = model5.predict(X_test1)



# DECISION TREE REGRESSOR

    model6 = DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                )
    


    model6.fit(X_train1, y_train)
    pred6 = model6.predict(X_test1)



    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[4,]]
    test = np.asarray(test).reshape(-1)

    r2 = r2_score(test,pred1)
    l.append(r2)
    r2 = r2_score(test,pred2)
    l.append(r2)    
    r2 = r2_score(test,pred3)
    l.append(r2)
    r2 = r2_score(test,pred4)
    l.append(r2)
    r2 = r2_score(test,pred5)
    l.append(r2)
    r2 = r2_score(test,pred6)
    l.append(r2)
    r2 = r2_score(test,pred)
    l.append(r2)
    r2 = max(l)

    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [237]:
optimizer5 = BayesianOptimization(f = black_box_function5, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer5.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.495     | 59.01     | 5.518     | 48.66     | 71.77     | 104.4     | 33.2      | 97.65     | 2.299     | 4.024     | 115.2     | 121.3     | 19.77     | 86.57     | 19.68     | 4.113     | 120.3     | 21.17     | 0.1933    | 1.839     | 193.5     | 93.61     | 95.15     | 124.6     | 1.866     | 0.1732    

In [238]:
print(optimizer5.max)

{'target': 0.495, 'params': {'leaf_size': 59.0108951704103, 'learning_rate': 5.5175992668396505, 'max_depth1': 48.66153363827933, 'max_depth2': 71.7667833737621, 'max_depth3': 104.3570525141879, 'max_depth4': 33.19733484147608, 'max_leaf_nodes': 97.65117102284793, 'max_leaf_nodes1': 2.2990522498203134, 'max_leaf_nodes2': 4.023858899067552, 'max_trials': 115.21872986066687, 'min_child_samples': 121.31977983318782, 'min_impurity_decrease': 19.768507460025308, 'min_samples': 86.57333708872378, 'min_samples_leaf1': 19.684612866330944, 'min_samples_leaf2': 4.113002586689276, 'min_samples_split': 120.27212089870612, 'min_split_gain': 21.168192696782146, 'min_weight_fraction_leaf': 0.1932856413218147, 'n_estimators1': 1.839041100698492, 'n_estimators2': 193.49794515713538, 'n_estimators3': 93.61466468797977, 'n_neighbors': 95.15284414424846, 'num_leaves': 124.57041232627225, 'p': 1.8662892985816986, 'stop_probability': 0.17316542149594816, 'subsample': 0.16745372831179633}}


In [239]:
parameters_cool = optimizer5.max["params"]
parameters_cool

{'leaf_size': 59.0108951704103,
 'learning_rate': 5.5175992668396505,
 'max_depth1': 48.66153363827933,
 'max_depth2': 71.7667833737621,
 'max_depth3': 104.3570525141879,
 'max_depth4': 33.19733484147608,
 'max_leaf_nodes': 97.65117102284793,
 'max_leaf_nodes1': 2.2990522498203134,
 'max_leaf_nodes2': 4.023858899067552,
 'max_trials': 115.21872986066687,
 'min_child_samples': 121.31977983318782,
 'min_impurity_decrease': 19.768507460025308,
 'min_samples': 86.57333708872378,
 'min_samples_leaf1': 19.684612866330944,
 'min_samples_leaf2': 4.113002586689276,
 'min_samples_split': 120.27212089870612,
 'min_split_gain': 21.168192696782146,
 'min_weight_fraction_leaf': 0.1932856413218147,
 'n_estimators1': 1.839041100698492,
 'n_estimators2': 193.49794515713538,
 'n_estimators3': 93.61466468797977,
 'n_neighbors': 95.15284414424846,
 'num_leaves': 124.57041232627225,
 'p': 1.8662892985816986,
 'stop_probability': 0.17316542149594816,
 'subsample': 0.16745372831179633}

# **Appliance 6 (Heating)**

In [240]:

def black_box_function6(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes
                       ):

    l = []
    y_train = y_train1[:,[5,]]
    y_train = np.asarray(y_train).reshape(-1)
# GRADIENT BOOSTING REGRESSOR

    model1 = GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           )
    
    model1.fit(X_train1, y_train)
    pred1 = model1.predict(X_test1)
    
    

# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train)
    pred2 = model2.predict(X_test1)


# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train)
    pred3 = model3.predict(X_test1)



# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train)
    pred4 = model4.predict(X_test1)



# LGBM REGRESSOR

    model5 = LGBMRegressor(num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                )
    


    model5.fit(X_train1, y_train)
    pred5 = model5.predict(X_test1)



# DECISION TREE REGRESSOR

    model6 = DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                )
    


    model6.fit(X_train1, y_train)
    pred6 = model6.predict(X_test1)



    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    test = y_test1[:,[5,]]
    test = np.asarray(test).reshape(-1)

    r2 = r2_score(test,pred1)
    l.append(r2)
    r2 = r2_score(test,pred2)
    l.append(r2)    
    r2 = r2_score(test,pred3)
    l.append(r2)
    r2 = r2_score(test,pred4)
    l.append(r2)
    r2 = r2_score(test,pred5)
    l.append(r2)
    r2 = r2_score(test,pred6)
    l.append(r2)
    r2 = r2_score(test,pred)
    l.append(r2)
    r2 = max(l)

    r2 = r2_score(test,pred) 
    r2 = round(r2, 3)
    return r2


In [241]:
optimizer6 = BayesianOptimization(f = black_box_function6, pbounds = pbounds, verbose = 2, random_state = 4)
optimizer6.maximize(init_points = 10, n_iter = 100)

|   iter    |  target   | leaf_size | learni... | max_de... | max_de... | max_de... | max_de... | max_le... | max_le... | max_le... | max_tr... | min_ch... | min_im... | min_sa... | min_sa... | min_sa... | min_sa... | min_sp... | min_we... | n_esti... | n_esti... | n_esti... | n_neig... | num_le... |     p     | stop_p... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.055     | 59.01     | 5.518     | 48.66     | 71.77     | 104.4     | 33.2      | 97.65     | 2.299     | 4.024     | 115.2     | 121.3     | 19.77     | 86.57     | 19.68     | 4.113     | 120.3     | 21.17     | 0.1933    | 1.839     | 193.5     | 93.61     | 95.15     | 124.6     | 1.866     | 0.1732    

In [242]:
print(optimizer6.max)

{'target': 0.055, 'params': {'leaf_size': 59.0108951704103, 'learning_rate': 5.5175992668396505, 'max_depth1': 48.66153363827933, 'max_depth2': 71.7667833737621, 'max_depth3': 104.3570525141879, 'max_depth4': 33.19733484147608, 'max_leaf_nodes': 97.65117102284793, 'max_leaf_nodes1': 2.2990522498203134, 'max_leaf_nodes2': 4.023858899067552, 'max_trials': 115.21872986066687, 'min_child_samples': 121.31977983318782, 'min_impurity_decrease': 19.768507460025308, 'min_samples': 86.57333708872378, 'min_samples_leaf1': 19.684612866330944, 'min_samples_leaf2': 4.113002586689276, 'min_samples_split': 120.27212089870612, 'min_split_gain': 21.168192696782146, 'min_weight_fraction_leaf': 0.1932856413218147, 'n_estimators1': 1.839041100698492, 'n_estimators2': 193.49794515713538, 'n_estimators3': 93.61466468797977, 'n_neighbors': 95.15284414424846, 'num_leaves': 124.57041232627225, 'p': 1.8662892985816986, 'stop_probability': 0.17316542149594816, 'subsample': 0.16745372831179633}}


In [243]:
parameters_heat = optimizer6.max["params"]
parameters_heat

{'leaf_size': 59.0108951704103,
 'learning_rate': 5.5175992668396505,
 'max_depth1': 48.66153363827933,
 'max_depth2': 71.7667833737621,
 'max_depth3': 104.3570525141879,
 'max_depth4': 33.19733484147608,
 'max_leaf_nodes': 97.65117102284793,
 'max_leaf_nodes1': 2.2990522498203134,
 'max_leaf_nodes2': 4.023858899067552,
 'max_trials': 115.21872986066687,
 'min_child_samples': 121.31977983318782,
 'min_impurity_decrease': 19.768507460025308,
 'min_samples': 86.57333708872378,
 'min_samples_leaf1': 19.684612866330944,
 'min_samples_leaf2': 4.113002586689276,
 'min_samples_split': 120.27212089870612,
 'min_split_gain': 21.168192696782146,
 'min_weight_fraction_leaf': 0.1932856413218147,
 'n_estimators1': 1.839041100698492,
 'n_estimators2': 193.49794515713538,
 'n_estimators3': 93.61466468797977,
 'n_neighbors': 95.15284414424846,
 'num_leaves': 124.57041232627225,
 'p': 1.8662892985816986,
 'stop_probability': 0.17316542149594816,
 'subsample': 0.16745372831179633}

#***Model Implementation with Optimum Parameters***

In [244]:
def bayes(learning_rate, min_samples_leaf1, n_estimators1, subsample, max_depth1,
                       max_leaf_nodes1, 
                       n_estimators2, max_depth2, min_samples_leaf2, max_leaf_nodes2,
                       n_neighbors, leaf_size, p,
                       min_samples, max_trials, stop_probability,
                       num_leaves, max_depth3, n_estimators3, min_split_gain, min_child_samples,
                       max_depth4, min_samples_split, min_weight_fraction_leaf, min_impurity_decrease, max_leaf_nodes,appliance
                       ):

    model_list = []
    y_train = y_train1[:,[appliance,]]
    y_train = np.asarray(y_train).reshape(-1)

# GRADIENT BOOSTING REGRESSOR

    model1 = GradientBoostingRegressor(learning_rate=round(learning_rate, 3),
                                                           min_samples_leaf=int(min_samples_leaf1), 
                                                           n_estimators=int(n_estimators1),
                                                           subsample=round(subsample,3),
                                                           max_depth = round(max_depth1,3),
                                                           max_leaf_nodes= int(max_leaf_nodes1)
                                                           )
    
    model1.fit(X_train1, y_train)
            
# RANDOM FOREST REGRESSOR

    model2 = RandomForestRegressor(n_estimators=int(n_estimators2),
                                   max_depth=round(max_depth2,3), 
                                   min_samples_leaf = int(min_samples_leaf2),
                                   max_leaf_nodes= int(max_leaf_nodes2)
                                                           )
    


    model2.fit(X_train1, y_train)

# K NEIGHBOR REGRESSOR

    model3 = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                 leaf_size = leaf_size,
                                 p=int(p)
                                 )
    


    model3.fit(X_train1, y_train)

# RANSAC REGRESSOR

    model4 = RANSACRegressor(min_samples=int(min_samples),
                                 max_trials = int(max_trials),
                                 stop_probability = round(stop_probability,3)
                                 )
    


    model4.fit(X_train1, y_train)

# LGBM REGRESSOR

    model5 = LGBMRegressor(num_leaves=int(num_leaves),
                                                max_depth = int(max_depth3),
                                                n_estimators=int(n_estimators3),
                                                min_split_gain = round(min_split_gain,3),
                                                min_child_samples = int(min_child_samples),
                                                max_leaf_nodes = int(max_leaf_nodes)
                                                
                                                )
    


    model5.fit(X_train1, y_train)


# DECISION TREE REGRESSOR

    model6 = DecisionTreeRegressor(
                                                max_depth = int(max_depth4),
                                                min_samples_split = int(min_samples_split),
                                                min_weight_fraction_leaf = round(min_weight_fraction_leaf,3),
                                                min_impurity_decrease = round(min_impurity_decrease,3),                                               
                                                )
    


    model6.fit(X_train1, y_train)

    model_list.append(model1)    
    model_list.append(model2)
    model_list.append(model3)
    model_list.append(model4)
    model_list.append(model5)
    model_list.append(model6)

    return model_list


In [245]:
def bayes_model(model_list,test):

    model1 = model_list[0]
    pred1=model1.predict(test)
    model2 = model_list[1]
    pred2=model2.predict(test)
    model3 = model_list[2]
    pred3=model3.predict(test)
    model4 = model_list[1]
    pred4=model4.predict(test)
    model5 = model_list[1]
    pred5=model5.predict(test)
    model6 = model_list[1]
    pred6=model6.predict(test)

    pred = (pred1+pred2+pred3+pred4+pred5+pred6)/6

    return pred        

In [246]:
def score(output,true,appliance):
  true = true[:,[appliance,]]
  true = np.asarray(true).reshape(-1)
  
  r2 = r2_score(true,output)
  mae = mean_absolute_error(true,output)

  return r2,mae

In [247]:
#############. Extracting from dictionary ################

from argparse import Namespace    


For estimating ventilation power

In [248]:
n1 = Namespace(**parameters_vent)
appliance = 0
vent = bayes(n1.learning_rate, n1.min_samples_leaf1, n1.n_estimators1, n1.subsample, n1.max_depth1,
                       n1.max_leaf_nodes1, 
                       n1.n_estimators2, n1.max_depth2, n1.min_samples_leaf2, n1.max_leaf_nodes2,
                       n1.n_neighbors, n1.leaf_size, n1.p,
                       n1.min_samples, n1.max_trials, n1.stop_probability,
                       n1.num_leaves, n1.max_depth3, n1.n_estimators3, n1.min_split_gain, n1.min_child_samples,
                       n1.max_depth4, n1.min_samples_split, n1.min_weight_fraction_leaf, n1.min_impurity_decrease, n1.max_leaf_nodes, appliance)

In [250]:
# this provide the output power of the ventilation appliance based on input
output = bayes_model(vent,X_test1)

In [251]:
# incase if the user already knows the output then this function can be used to compare machine's output with the actual output through the scores of evaluation metrics 
scores = score(output,y_test1,appliance)

#  score[0] ----> will give r2 score 
#  and
#. score[1] ----> will give mae score 

For estimating socket plugs power

In [252]:
n2 = Namespace(**parameters_soc)
appliance = 1
soc = bayes(n2.learning_rate, n2.min_samples_leaf1, n2.n_estimators1, n2.subsample, n2.max_depth1,
                       n2.max_leaf_nodes1, 
                       n2.n_estimators2, n2.max_depth2, n2.min_samples_leaf2, n2.max_leaf_nodes2,
                       n2.n_neighbors, n2.leaf_size, n2.p,
                       n2.min_samples, n2.max_trials, n2.stop_probability,
                       n2.num_leaves, n2.max_depth3, n2.n_estimators3, n2.min_split_gain, n2.min_child_samples,
                       n2.max_depth4, n2.min_samples_split, n2.min_weight_fraction_leaf, n2.min_impurity_decrease, n2.max_leaf_nodes, appliance)



In [253]:
# this provide the output power of the socket plug appliance
output = bayes_model(soc,X_test1)

In [254]:
# incase if the user already knows the output then this function can be used to compare machine's output with the actual output through the scores of evaluation metrics 
scores = score(output,y_test1,appliance)

#  score[0] ----> will give r2 score 
#  and
#. score[1] ----> will give mae score 

For estimating Lighting power

In [255]:
n3 = Namespace(**parameters_light)
appliance = 2
light = bayes(n3.learning_rate, n3.min_samples_leaf1, n3.n_estimators1, n3.subsample, n3.max_depth1,
                      n3.max_leaf_nodes1,
                      n3.n_estimators2, n3.max_depth2, n3.min_samples_leaf2, n3.max_leaf_nodes2,
                      n3.n_neighbors, n3.leaf_size, n3.p,
                      n3.min_samples, n3.max_trials, n3.stop_probability,
                      n3.num_leaves, n3.max_depth3, n3.n_estimators3, n3.min_split_gain, n3.min_child_samples,
                      n3.max_depth4, n3.min_samples_split, n3.min_weight_fraction_leaf, n3.min_impurity_decrease, n3.max_leaf_nodes, appliance)

In [256]:
# this provide the output power of the socket plug appliance
output = bayes_model(light,X_test1)

In [257]:
# incase if the user already knows the output then this function can be used to compare machine's output with the actual output through the scores of evaluation metrics 
scores = score(output,y_test1,appliance)

#  score[0] ----> will give r2 score 
#  and
#. score[1] ----> will give mae score 

For estimating Other Electricity power

In [258]:
n4 = Namespace(**parameters_other)
appliance = 3
other = bayes(n4.learning_rate, n4.min_samples_leaf1, n4.n_estimators1, n4.subsample, n4.max_depth1,
                     n4.max_leaf_nodes1,
                     n4.n_estimators2, n4.max_depth2, n4.min_samples_leaf2, n4.max_leaf_nodes2,
                     n4.n_neighbors, n4.leaf_size, n4.p,
                     n4.min_samples, n4.max_trials, n4.stop_probability,
                     n4.num_leaves, n4.max_depth3, n4.n_estimators3, n4.min_split_gain, n4.min_child_samples,
                     n4.max_depth4, n4.min_samples_split, n4.min_weight_fraction_leaf, n4.min_impurity_decrease, n4.max_leaf_nodes, appliance)

In [259]:
# this provide the output power of the socket plug appliance
output = bayes_model(other,X_test1)

In [260]:
# incase if the user already knows the output then this function can be used to compare machine's output with the actual output through the scores of evaluation metrics 
scores = score(output,y_test1,appliance)

#  score[0] ----> will give r2 score 
#  and
#. score[1] ----> will give mae score 

For estimating Cooling power

In [261]:
n5 = Namespace(**parameters_cool)
appliance = 4
cool = bayes(n5.learning_rate, n5.min_samples_leaf1, n5.n_estimators1, n5.subsample, n5.max_depth1,
                    n5.max_leaf_nodes1,
                    n5.n_estimators2, n5.max_depth2, n5.min_samples_leaf2, n5.max_leaf_nodes2,
                    n5.n_neighbors, n5.leaf_size, n5.p,
                    n5.min_samples, n5.max_trials, n5.stop_probability,
                    n5.num_leaves, n5.max_depth3, n5.n_estimators3, n5.min_split_gain, n5.min_child_samples,
                    n5.max_depth4, n5.min_samples_split, n5.min_weight_fraction_leaf, n5.min_impurity_decrease, n5.max_leaf_nodes, appliance)

In [263]:
# this provide the output power of the socket plug appliance
output = bayes_model(cool,X_test1)

In [266]:
# incase if the user already knows the output then this function can be used to compare machine's output with the actual output through the scores of evaluation metrics 
scores = score(output,y_test1,appliance)

#  score[0] ----> will give r2 score 
#  and
#. score[1] ----> will give mae score 


For estimating Heating power

In [267]:
n6 = Namespace(**parameters_heat)
appliance = 5
heat = bayes(n6.learning_rate, n6.min_samples_leaf1, n6.n_estimators1, n6.subsample, n6.max_depth1,
                   n6.max_leaf_nodes1,
                   n6.n_estimators2, n6.max_depth2, n6.min_samples_leaf2, n6.max_leaf_nodes2,
                   n6.n_neighbors, n6.leaf_size, n6.p,
                   n6.min_samples, n6.max_trials, n6.stop_probability,
                   n6.num_leaves, n6.max_depth3, n6.n_estimators3, n6.min_split_gain, n6.min_child_samples,
                   n6.max_depth4, n6.min_samples_split, n6.min_weight_fraction_leaf, n6.min_impurity_decrease, n6.max_leaf_nodes, appliance)

In [270]:
# this provide the output power of the socket plug appliance
output = bayes_model(heat,X_test1)

In [271]:
# incase if the user already knows the output then this function can be used to compare machine's output with the actual output through the scores of evaluation metrics 
scores = score(output,y_test1,appliance)

#  score[0] ----> will give r2 score 
#  and
#. score[1] ----> will give mae score 